In [1]:
%pwd

'e:\\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS'

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import sentence_transformers

import langchain

e:\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS\llmbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install -q langchain

print(f'Langchain Version: {langchain.__version__}')
import sys
print(f'Python Version: {sys.version}')

Langchain Version: 1.2.8
Python Version: 3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]


In [6]:
# Extract the Data From PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob= "*pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data= load_pdf_file(data='Data/')

Ignoring wrong pointing object 0 0 (offset 0)


In [8]:
extracted_data

[Document(metadata={'producer': 'Adobe Acrobat 6.0', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2004-05-31T18:06:21+04:00', 'moddate': '2004-05-31T18:06:49+04:00', 'source': 'Data\\The GALE Encyclopedia of Science, 3d edition, Vol2(792s).pdf', 'total_pages': 792, 'page': 0, 'page_label': '1'}, page_content='The GALE\nENCYCLOPEDIA\nof Science\nTHIRD EDITION\ntphtp_v2  10/13/03  10:45 AM  Page 1'),
 Document(metadata={'producer': 'Adobe Acrobat 6.0', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2004-05-31T18:06:21+04:00', 'moddate': '2004-05-31T18:06:49+04:00', 'source': 'Data\\The GALE Encyclopedia of Science, 3d edition, Vol2(792s).pdf', 'total_pages': 792, 'page': 1, 'page_label': '2'}, page_content='tphtp_v2  10/13/03  10:45 AM  Page 2'),
 Document(metadata={'producer': 'Adobe Acrobat 6.0', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2004-05-31T18:06:21+04:00', 'moddate': '2004-05-31T18:06:49+04:00', 'source': 'Data\\The GALE Encyclopedia of Science, 3d edition, Vol2(7

In [9]:
# Split the Data into the Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of Text chunks", len(text_chunks))

Length of Text chunks 7683


In [11]:
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2") 

In [12]:

embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2") 


C:\Users\ragha\AppData\Local\Temp\ipykernel_15696\2622195746.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
Loading weights: 100%|██████████| 391/391 [00:00<00:00, 690.86it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: intfloat/e5-large-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [13]:
query_result = embeddings.embed_query("hello world")
print("Length", len(query_result))

Length 1024


In [14]:
query_result

[0.01732616312801838,
 -0.036889512091875076,
 0.030519118532538414,
 -0.011622456833720207,
 -0.039458442479372025,
 0.013219408690929413,
 -0.04048957675695419,
 -0.02140515111386776,
 -0.04637158289551735,
 0.005944640375673771,
 0.03787526488304138,
 0.0033623254857957363,
 0.04230577126145363,
 0.04927029833197594,
 -0.019306054338812828,
 0.03237044811248779,
 0.0018610036931931973,
 -0.003486003028228879,
 0.013467535376548767,
 -0.015579435043036938,
 0.03057544305920601,
 -0.02736688405275345,
 0.021773019805550575,
 0.029492394998669624,
 -0.023842258378863335,
 -0.023290403187274933,
 -0.028788337484002113,
 -0.03102944605052471,
 -0.028318697586655617,
 0.0194327961653471,
 -0.00505313603207469,
 -0.00740543007850647,
 0.013269439339637756,
 -0.01984167844057083,
 0.029006240889430046,
 -0.014556915499269962,
 -0.018121670931577682,
 -0.062444377690553665,
 0.00521544786170125,
 0.005411810707300901,
 0.009714444167912006,
 0.04413551837205887,
 -0.026988312602043152,
 0.03

# VectorDB(PINECONE) Connection

In [15]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY = api_key=os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = api_key=os.environ.get("OPENAI_API_KEY")

In [17]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "llm-med-bot"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY   

In [19]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Embed each chunk and upsert the  embeddings into your Pinecone index

from langchain_pinecone.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)


In [38]:
import os

PINECONE_API_KEY = api_key=os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY= api_key=os.environ.get("OPENAI_API_KEY")

In [39]:
from langchain_pinecone.vectorstores import Pinecone

# Embed each chunk and upsert the embeddings into your Pinecone Index
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [40]:
docsearch

In [41]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [25]:
# Example of input and output of Semantic search

retrieved_docs = retriever.invoke("whats is the Acne")

In [42]:
retrieved_docs

[Document(id='e8de7b6d-23cb-4850-9de4-fe164bce9e42', metadata={'creationdate': '2004-05-31T18:06:21+04:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2004-05-31T18:06:49+04:00', 'page': 6.0, 'page_label': '7', 'producer': 'Adobe Acrobat 6.0', 'source': 'Data\\The GALE Encyclopedia of Science, 3d edition, Vol2(792s).pdf', 'total_pages': 792.0}, page_content='A\nAardvark\nAbacus\nAbrasives\nAbscess\nAbsolute zero\nAbyssal plain\nAcceleration\nAccelerators\nAccretion disk\nAccuracy\nAcetic acid\nAcetone\nAcetylcholine\nAcetylsalicylic acid\nAcid rain\nAcids and bases\nAcne\nAcorn worm\nAcoustics\nActinides\nAction potential\nActivated complex\nActive galactic nuclei\nAcupressure\nAcupuncture\nADA (adenosine deaminase)\ndeficiency\nAdaptation\nAddiction\nAddison’s disease\nAddition\nAdenosine diphosphate\nAdenosine triphosphate\nAdhesive\nAlluvial systems\nAlpha particle'),
 Document(id='b46f52ee-b219-4eb2-91a6-df67a7f46206', metadata={'creationdate': '2004-05-31T18:06:21+04:00', 'creato

# LLM (OpenAi) connection

In [43]:
from langchain_openai import OpenAI
llm =OpenAI(temperature=0.4, max_tokens=500, api_key=os.environ.get("OPENAI_API_KEY"))

In [52]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2", temperature=0.4)

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [53]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer the question at the end."
    "the question that if you dont know the answer say that you"
    "dont know . Use these senteences maximum and keep the answer concise ."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [54]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

## Test cases input - output and the result of LLM

In [58]:
response = rag_chain.invoke({"input":"What is Acne and how is it treated?"})
print(response['answer'])

Acne is a skin condition characterized by inflammation, redness, and comedones (blackheads or whiteheads). It occurs when the pores on the skin become clogged with dead skin cells, oil, and bacteria.

Treatment for acne typically involves:

1. Topical treatments: applying creams or gels containing benzoyl peroxide, salicylic acid, or sulfacetamide to reduce inflammation and kill bacteria.
2. Oral antibiotics: taking medications like doxycycline or minocycline to target bacterial growth.
3. Hormonal treatments: using birth control pills or spironolactone to regulate hormone levels.
4. Isotretinoin: a powerful oral retinoid medication for severe cases of acne.

It's essential to consult a dermatologist for proper diagnosis and treatment, as over-the-counter products may not be effective for everyone.


In [59]:
response = rag_chain.invoke({"input":"What is GEN-AI?"})
print(response['answer'])

I can help answer your question about GEN-AI. Unfortunately, I don't have information on what GEN-AI refers to.
